### Imports

In [1]:
from tqdm import tqdm
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_mtsim
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
from gym_mtsim_forked.gym_mtsim.data import FOREX_DATA_PATH_TRAIN, FOREX_DATA_PATH_TEST, FOREX_DATA_PATH
from gym_mtsim import OrderType, Timeframe, MtEnv, MtSimulator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, STATUS_FAIL
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import time
import torch

c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-unhedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-unhedge-v0 alr

# Parameters

In [2]:
# import pickle
# import time
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


# def objective(x):
#     return {
#         'loss': x ** 2,
#         'status': STATUS_OK,
#         # -- store other results like this
#         'eval_time': time.time(),
#         'other_stuff': {'type': None, 'value': x},
#         # -- attachments are handled differently
#         'attachments':
#             {'time_module': pickle.dumps(time.time)}
#         }
# trials = Trials()
# best = fmin(objective,
#             space=hp.uniform('x', -10, 10),
#             algo=tpe.suggest,
#             max_evals=100,
#             trials=trials)

# print(best)

In [3]:
# trials.results

In [30]:
import pickle
# unpack the pickle file and load the data that is in symbols_forex.pkl
with open('C:/Users/WilliamFetzner/Documents/Trading/gym_mtsim_forked/gym_mtsim/data/symbols_forex.pkl', 'rb') as f:
    symbols = pickle.load(f)
# convert symbols to a pd.dataframe
# symbols[1]['EURUSD']
split = int(len(symbols[1]['EURUSD']) * 0.80)
validation_split = int(len(symbols[1]['EURUSD']) * 0.90)

In [35]:
# get the 2 weeks of the symbols[1]['EURUSD'] dataframe by first finding the max date
# then subtracting 14 days from that date
symbols[1]['EURUSD'].index = pd.to_datetime(symbols[1]['EURUSD'].index)
max_date = symbols[1]['EURUSD'].index.max()
two_weeks = max_date - pd.DateOffset(days=14)
one_week = max_date - pd.DateOffset(days=7)

In [36]:
training_index_slice = symbols[1]['EURUSD'].loc[:two_weeks, :].index
validation_index_slice = symbols[1]['EURUSD'].loc[two_weeks:one_week, :].index
testing_index_slice = symbols[1]['EURUSD'].loc[one_week:, :].index

In [39]:
validation_index_slice

DatetimeIndex(['2024-04-03 00:00:00+00:00', '2024-04-03 01:00:00+00:00',
               '2024-04-03 02:00:00+00:00', '2024-04-03 03:00:00+00:00',
               '2024-04-03 04:00:00+00:00', '2024-04-03 05:00:00+00:00',
               '2024-04-03 06:00:00+00:00', '2024-04-03 07:00:00+00:00',
               '2024-04-03 08:00:00+00:00', '2024-04-03 09:00:00+00:00',
               ...
               '2024-04-09 15:00:00+00:00', '2024-04-09 16:00:00+00:00',
               '2024-04-09 17:00:00+00:00', '2024-04-09 18:00:00+00:00',
               '2024-04-09 19:00:00+00:00', '2024-04-09 20:00:00+00:00',
               '2024-04-09 21:00:00+00:00', '2024-04-09 22:00:00+00:00',
               '2024-04-09 23:00:00+00:00', '2024-04-10 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Time', length=121, freq=None)

### Create Env

In [7]:
sim_train = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_train = MtEnv(
    original_simulator=sim_train,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(training_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [8]:
sim_validation = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_validation = MtEnv(
    original_simulator=sim_validation,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(validation_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [9]:
sim_testing = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_testing = MtEnv(
    original_simulator=sim_testing,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(testing_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

### Define Functions

In [10]:
def print_stats(reward_over_episodes, printing_name):
    """  Print Reward  """

    avg_rewards = np.mean(reward_over_episodes)
    min_rewards = np.min(reward_over_episodes)
    max_rewards = np.max(reward_over_episodes)

    print (f'Min. {printing_name}          : {min_rewards:>10.3f}')
    print (f'Avg. {printing_name}          : {avg_rewards:>10.3f}')
    print (f'Max. {printing_name}          : {max_rewards:>10.3f}')

    return min_rewards, avg_rewards, max_rewards


# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


In [11]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, -2), # Learning rate
    # 'gamma': hp.uniform('gamma', 0.97, 0.99), # Discount factor
    'ent_coef': hp.loguniform('ent_coef', -5, 0), # Entropy coefficient
    # 'learning_timesteps': hp.choice('learning_timesteps', [25, 50, 100, 250, 500]),
    'window_size': hp.choice('window_size', [10, 20, 50]) # Window size
}

In [12]:
# TRAINING + TEST
def train_val_model(model, model_policy, env_tr, env_val, seed, steps_str, window_size_param, lr, gamma_param, entropy, total_learning_timesteps=10_000):
    """
    Trains and validates a model using the Proximal Policy Optimization (PPO) algorithm.

    Args:
        model (object): The model to be trained.
        model_policy (object): The policy used by the model.
        env_tr (object): The training environment.
        env_val (object): The validation environment.
        seed (int): The random seed for reproducibility.
        steps_str (str): A string representing the number of steps.
        window_size_param (int): The window size parameter.
        lr (float): The learning rate.
        gamma_param (float): The gamma parameter.
        entropy (float): The entropy coefficient.
        total_learning_timesteps (int, optional): The total number of learning timesteps. Defaults to 10,000.

    Returns:
        tuple: A tuple containing the reward over validations, orders over validations, and the model dictionary.
    """
    # reproduce training and test
    print('-' * 80)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    #model_dict = {}
    env_tr.window_size = window_size_param
    print(f'Window size: {window_size_param}, entropy: {entropy}, learning rate: {lr}')
    # eval_callback = EvalCallback(env_tr, log_path='./logs/', eval_freq=1000)
    model = PPO(model_policy, env_tr, verbose=0, ent_coef=entropy, learning_rate=lr)#, gamma=gamma_param, 
    obs_tr, info_tr = env_tr.reset(seed=seed)
    # custom callback for 'progress_bar'
    model.learn(total_timesteps=total_learning_timesteps)#, callback=ProgressBarCallback(100))

    reward_over_validations = []
    orders_over_validations = []

    tbar = tqdm(range(0, 10))

    for episode in tbar:
        env_val.window_size = window_size_param
        obs_val, info_val = env_val.reset(seed=seed)

        total_reward = 0
        done_val = False

        while not done_val:
            action, _states = model.predict(obs_val)
            obs_val, reward_val, terminated_val, truncated_val, info_val = env_val.step(action)
            done_val = terminated_val or truncated_val

            total_reward += reward_val
            if done_val:
                break
        try:
            order_len = len(env_val.render()['orders'])
        except:
            order_len = 0

        # model_dict[f'model_{episode}'] = model
        # model.save(f'models_4_19_24/window_{window_size_param}_entropy_{round(entropy, 4)}/model_{steps_str}_{episode}.pkl')

        reward_over_validations.append(total_reward)    
        orders_over_validations.append(order_len)  


        # if episode % 1 == 0:
        avg_reward = np.mean(reward_over_validations)
        avg_orders = np.mean(orders_over_validations)
        tbar.set_description(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}, # of orders: {avg_orders:.3f}')
        tbar.update()

    tbar.close()
    avg_reward = np.mean(reward_over_validations)

    return reward_over_validations, orders_over_validations#, model_dict

### Train + Test Env

In [13]:
seed = 2024  # random seed
total_num_episodes = 10

# print ("env_name                 :", env_name)
print ("seed                     :", seed)

# INIT matplotlib
plot_settings = {}
plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}

# learning_timesteps_list_in_K = [25]#, 50, 100]
# learning_timesteps_list_in_K = [50, 250, 500]
# learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

timesteps_models_dict = {}
def objective(params):
    window_size = params['window_size']
    learning_timesteps = 50 #params['learning_timesteps']
    ent_coef = params['ent_coef']
    gamma = 0.99 #params['gamma']
    learning_rate = params['learning_rate']#0.0003#

    total_learning_timesteps = learning_timesteps * 1000
    step_key = f'{learning_timesteps}K'
    policy_dict = PPO.policy_aliases
    policy = policy_dict.get('MultiInputPolicy')
    class_name = type(PPO).__qualname__
    plot_key = f'{class_name}_rewards_'+step_key
    try:
        rewards, orders = train_val_model(PPO, policy, env_train, env_validation, seed, step_key, window_size, 
                                                    learning_rate, gamma, ent_coef, total_learning_timesteps)
    except:
        print(f'''there was an error with those parameters: Window: {window_size}, timesteps: {learning_timesteps}, \n
              ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
    # timesteps_models_dict[step_key] = models_dict
    min_rewards, avg_rewards, max_rewards, = print_stats(rewards, 'Reward')
    print_stats(orders, 'Orders')
    label = f'Avg. {avg_rewards:>7.2f} : {class_name} - {step_key}'
    plot_data[plot_key] = rewards
    plot_settings[plot_key] = {'label': label}     

    return {'loss': -avg_rewards, 'status': STATUS_OK, 'eval_time': time.time(), 'parameters': params} 

seed                     : 2024


In [14]:
# # check if it is working:
# parameters = {
#     'window_size': 10,
#     'learning_timesteps': 25,
#     'ent_coef': 0.021594542314204903,
#     'gamma': 0.9484679718228304,
#     'learning_rate': 0.0661452347963559
# }
# objective(parameters)

In [15]:
# PPO('MultiInputPolicy', env_train, verbose=0, ent_coef=parameters['ent_coef']).learn(total_timesteps=25_000) #, learning_rate=parameters['learning_rate'], gamma=parameters['gamma'], ent_coef=parameters['ent_coef']

In [16]:
trials = Trials()
best = fmin(fn=objective,
             space=space,
             algo=tpe.suggest,
             max_evals=100, # Number of evaluations of the objective function
             trials=trials,
             trials_save_file='hyperopt/trials.pkl')

print("Best parameters:", best)

--------------------------------------------------------------------------------
Window size: 50, entropy: 0.05787549768378316, learning rate: 0.06685483129710154
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

there was an error with those parameters: Window: 50, timesteps: 50, 

              ent_coef: 0.05787549768378316, gamma: 0.99, learning_rate: 0.06685483129710154
--------------------------------------------------------------------------------
Window size: 20, entropy: 0.13718083236225725, learning rate: 0.011962918902699305
there was an error with those parameters: Window: 20, timesteps: 50, 

              ent_coef: 0.13718083236225725, gamma: 0.99, learning_rate: 0.011962918902699305
--------------------------------------------------------------------------------
Window size: 10, entropy: 0.024532025239255142, learning rate: 0.04654130844370433
  2%|▏         | 2/100 [01:58<1:37:33, 59.73s/trial, best loss=?]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -97378.900, # of orders: 787.000:   0%|          | 0/10 [00:07<?, ?it/s]
Episode: 0, Avg. Reward: -97378.900, # of orders: 787.000:  10%|#         | 1/10 [00:07<01:11,  7.99s/it]
Episode: 1, Avg. Reward: -101957.629, # of orders: 434.500:  20%|##        | 2/10 [00:26<01:03,  7.99s/it]
Episode: 1, Avg. Reward: -101957.629, # of orders: 434.500:  30%|###       | 3/10 [00:26<01:03,  9.00s/it]
Episode: 2, Avg. Reward: -105398.549, # of orders: 300.667:  40%|####      | 4/10 [00:45<00:54,  9.00s/it]
Episode: 2, Avg. Reward: -105398.549, # of orders: 300.667:  50%|#####     | 5/10 [00:45<00:45,  9.18s/it]
Episode: 3, Avg. Reward: -105044.694, # of orders: 227.500:  50%|#####     | 5/10 [01:05<00:45,  9.18s/it]
Episode: 3, Avg. Reward: -105044.694, # of orders: 227.500:  60%|######    | 6/10 [01:05<00:48, 12.16s/it]
Episode: 4, Avg. Reward: -103493.773, # of orders: 188.000:  70%|#######   | 7/10 [01:24<00:36, 12.16s/it]
Episode:

Min. Reward          : -149875.628                               
Avg. Reward          : -108189.443                               
Max. Reward          : -97290.089                                
Min. Orders          :      8.000                                
Avg. Orders          :    213.300                                
Max. Orders          :    787.000                                
--------------------------------------------------------------------------------    
Window size: 10, entropy: 0.041877558036327776, learning rate: 0.023286042267958402 
  3%|▎         | 3/100 [09:17<6:16:24, 232.83s/trial, best loss: 108189.44290358588]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -120962.782, # of orders: 145.000:   0%|          | 0/10 [00:16<?, ?it/s]
Episode: 0, Avg. Reward: -120962.782, # of orders: 145.000:  10%|#         | 1/10 [00:16<02:29, 16.64s/it]
Episode: 1, Avg. Reward: -110856.562, # of orders: 86.500:  20%|##        | 2/10 [00:34<02:13, 16.64s/it] 
Episode: 1, Avg. Reward: -110856.562, # of orders: 86.500:  30%|###       | 3/10 [00:34<01:15, 10.77s/it]
Episode: 2, Avg. Reward: -114025.811, # of orders: 72.000:  40%|####      | 4/10 [00:49<01:04, 10.77s/it]
Episode: 2, Avg. Reward: -114025.811, # of orders: 72.000:  50%|#####     | 5/10 [00:49<00:46,  9.24s/it]
Episode: 3, Avg. Reward: -113053.382, # of orders: 58.250:  50%|#####     | 5/10 [01:07<00:46,  9.24s/it]
Episode: 3, Avg. Reward: -113053.382, # of orders: 58.250:  60%|######    | 6/10 [01:07<00:45, 11.46s/it]
Episode: 4, Avg. Reward: -114199.264, # of orders: 54.200:  70%|#######   | 7/10 [01:26<00:34, 11.46s/it]
Episode: 4, 

...

Min. Reward          : -151166.863                                                   
Avg. Reward          : -108896.116                                                   
Max. Reward          : -96505.468                                                    
Min. Orders          :     11.000                                                    
Avg. Orders          :    175.400                                                    
Max. Orders          :    590.000                                                    
--------------------------------------------------------------------------------     
Window size: 50, entropy: 0.008763068538996601, learning rate: 0.035576367778539154  
there was an error with those parameters: Window: 50, timesteps: 50,                 

              ent_coef: 0.008763068538996601, gamma: 0.99, learning_rate: 0.035576367778539154
--------------------------------------------------------------------------------     
Window size: 20, entropy: 0.0730231968120579

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -182807.710, # of orders: 792.000:   0%|          | 0/10 [00:15<?, ?it/s]
Episode: 0, Avg. Reward: -182807.710, # of orders: 792.000:  10%|#         | 1/10 [00:15<02:23, 15.94s/it]
Episode: 1, Avg. Reward: -185601.100, # of orders: 863.500:  20%|##        | 2/10 [00:32<02:07, 15.94s/it]
Episode: 1, Avg. Reward: -185601.100, # of orders: 863.500:  30%|###       | 3/10 [00:32<01:11, 10.23s/it]
Episode: 2, Avg. Reward: -188834.499, # of orders: 811.333:  40%|####      | 4/10 [00:49<01:01, 10.23s/it]
Episode: 2, Avg. Reward: -188834.499, # of orders: 811.333:  50%|#####     | 5/10 [00:49<00:47,  9.44s/it]
Episode: 3, Avg. Reward: -190310.721, # of orders: 795.250:  50%|#####     | 5/10 [01:06<00:47,  9.44s/it]
Episode: 3, Avg. Reward: -190310.721, # of orders: 795.250:  60%|######    | 6/10 [01:06<00:45, 11.30s/it]
Episode: 4, Avg. Reward: -191274.570, # of orders: 789.200:  70%|#######   | 7/10 [01:22<00:33, 11.30s/it]
Episod

Min. Reward          : -198346.228                                                   
Avg. Reward          : -192103.143                                                   
Max. Reward          : -182807.710                                                   
Min. Orders          :    285.000                                                    
Avg. Orders          :    897.100                                                    
Max. Orders          :   1622.000                                                    
--------------------------------------------------------------------------------     
Window size: 10, entropy: 0.007242299364287828, learning rate: 0.018873441051237685  
 98%|█████████▊| 98/100 [8:27:03<08:13, 246.63s/trial, best loss: -3401.998593375756]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -185815.514, # of orders: 921.000:   0%|          | 0/10 [00:16<?, ?it/s]
Episode: 0, Avg. Reward: -185815.514, # of orders: 921.000:  10%|#         | 1/10 [00:16<02:24, 16.07s/it]
Episode: 1, Avg. Reward: -189423.227, # of orders: 733.500:  20%|##        | 2/10 [00:32<02:08, 16.07s/it]
Episode: 1, Avg. Reward: -189423.227, # of orders: 733.500:  30%|###       | 3/10 [00:32<01:11, 10.20s/it]
Episode: 2, Avg. Reward: -190672.038, # of orders: 688.000:  40%|####      | 4/10 [00:48<01:01, 10.20s/it]
Episode: 2, Avg. Reward: -190672.038, # of orders: 688.000:  50%|#####     | 5/10 [00:48<00:44,  9.00s/it]
Episode: 3, Avg. Reward: -190416.305, # of orders: 586.250:  50%|#####     | 5/10 [01:04<00:44,  9.00s/it]
Episode: 3, Avg. Reward: -190416.305, # of orders: 586.250:  60%|######    | 6/10 [01:04<00:43, 10.88s/it]
Episode: 4, Avg. Reward: -189497.033, # of orders: 587.400:  70%|#######   | 7/10 [01:19<00:32, 10.88s/it]
Episod

Min. Reward          : -197156.666                                                   
Avg. Reward          : -189639.527                                                   
Max. Reward          : -180509.776                                                   
Min. Orders          :    281.000                                                    
Avg. Orders          :    593.200                                                    
Max. Orders          :    921.000                                                    
--------------------------------------------------------------------------------     
Window size: 10, entropy: 0.08802768010237469, learning rate: 0.015367012631756488   
there was an error with those parameters: Window: 10, timesteps: 50,                 

              ent_coef: 0.08802768010237469, gamma: 0.99, learning_rate: 0.015367012631756488
100%|██████████| 100/100 [8:38:40<00:00, 311.21s/trial, best loss: -3401.998593375756]
Best parameters: {'ent_coef': 0.006745340429

In [43]:
results_df = pd.DataFrame()
for result in trials.results:
    new_row = pd.DataFrame(result, index=[0])
    results_df = pd.concat([results_df, new_row], axis=0)
results_df

,loss,status,eval_time,parameters
0,NaN,fail,1.713532e+09,NaN
0,NaN,fail,1.713532e+09,NaN
0,108189.442904,ok,1.713533e+09,NaN
0,112597.585936,ok,1.713533e+09,NaN
0,NaN,fail,1.713533e+09,NaN
...,...,...,...,...
0,NaN,fail,1.713562e+09,NaN
0,NaN,fail,1.713562e+09,NaN
0,192103.142642,ok,1.713563e+09,NaN
0,189639.526575,ok,1.713563e+09,NaN


### Plot Results

In [17]:
# # create a dataframe of the rewards
# rewards_df = pd.DataFrame({'rewards': rewards})
# # plot the rewards
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=rewards_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()


In [18]:
# # import the models from /models folder
# import os
# import glob
# # get the list of models
# model_list = glob.glob('models_4_17_24/*.pkl')
# # separate the strings of each model name on _ and get the last element of the string if the string of the model doesn't include 'initial' or 'updated'
# model_list_episode_nbr = [model.split('_')[-1] for model in model_list if 'initial' not in model and 'updated' not in model]
# model_list_episode_nbr = [int(model_name.split('.')[0]) for model_name in model_list_episode_nbr]
# max_episode = max(model_list_episode_nbr)
# # test the last set of 10 episodes
# init_episode = ((int(max_episode)/10) - 10)*10
# # print(max_episode, init_episode)
# models = []
# # test the last set of 10 episodes from init_episode to max_episode
# for nbr in range(int(init_episode), int(max_episode)+10, 10):
#     # set up the appropriate time_points for each of the models in the list
#     env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(nbr)), :].index)# make this -nbr not +nbr next time
#     obs_train, info_train = env_train.reset(seed=2024)
#     # find the model name that contains the nbr
#     model_name = [model for model in model_list if str(nbr) in model][0]
#     print(model_name)
#     # load the models into a list
#     models.append(PPO.load(model_name, env=env_train))

In [19]:
# sim_testing = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

# env_testing = MtEnv(
#     original_simulator=sim_testing,
#     trading_symbols=['EURUSD'],
#     window_size = window_size_param,
#     time_points=list(testing_index_slice),
#     hold_threshold=0.1,
#     close_threshold=0.1,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [20]:
# model_ppo = PPO.load(f'models_4_17_24\model_25K_5.pkl', env=env_train)

# obs_test, info_test = env_testing.reset(seed=2024)
# done_test = False
# while not done_test:
#     action, _states = model_ppo.predict(obs_test)
#     obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#     done_test = terminated_test or truncated_test
#     # total_reward += reward_test
#     if done_test:
#         break
# try:
#     order_len = len(env_testing.render()['orders'])
# except:
#     order_len = 0
# # print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")


In [21]:
# # if model_dict is still a thing
# for timestep in timesteps_models_dict.keys():
#     models_dict = timesteps_models_dict[timestep]

#     for nbr in range(0, 10):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in range(0, 10):   
#             total_reward = 0
#             done_test = False
#             model_ppo = models_dict[f'model_{nbr}']

#             obs_test, info_test = env_train.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_train.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_train.render()['orders'])
#             except:
#                 order_len = 0
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes)
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [22]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [23]:
# # if model_dict is still a thing
# for timestep in ['25K']:
#     for nbr in tqdm(range(1, 10)):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         number_of_orders_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in tqdm(range(0, 10)):   
#             total_reward = 0
#             done_test = False
#             model_ppo = PPO.load(f'models_4_17_24\model_{timestep}_{nbr}.pkl', env=env_train)

#             obs_test, info_test = env_testing.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_testing.render()['orders'])
#             except:
#                 order_len = 0
#             number_of_orders_across_episodes.append(order_len)
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes, 'Reward')
#         print_stats(number_of_orders_across_episodes, 'Orders')
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [24]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [25]:
# # if the cluster has been restarted
# for nbr, time_points_idx in zip(range(0, 10), range(0, 500, 50)):
#     # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))
#     env_train = MtEnv(
#         original_simulator=sim_train,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         time_points=list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(time_points_idx)), :].index),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     # obs_train, info_train = env_train.reset(seed=2024)
#     total_reward = 0
#     done_test = False
#     model_ppo = PPO.load(f'models\model_{nbr}.pkl', env=env_train)

#     env_testing = MtEnv(
#         original_simulator=sim_testing,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         # time_points=list(testing_index_slice),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     obs_test, info_test = env_testing.reset(seed=2024)
#     while not done_test:
#         action, _states = model_ppo.predict(obs_test)
#         obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#         done_test = terminated_test or truncated_test
#         total_reward += reward_test
#         if done_test:
#             break
#     state = env_testing.render()

#     print(
#         f"balance: {state['balance']}, equity: {state['equity']}, margin: {state['margin']}\n"
#         f"free_margin: {state['free_margin']}, margin_level: {state['margin_level']}\n"

#     )
#     # print(state['orders'].Profit.sum())
#     if len(state['orders']) > 0:
#         print(state['orders'].Profit.sum())

In [26]:
# max_episode = 120
# for model_nbr in range(0, int(max_episode)+10, 10):
#     print(f'Model: {model_nbr}')
#     over_episodes_rewards = []
#     over_episodes_balance = []
#     obs_training, info_training = env_train.reset(seed=2024)
#     for episode in range(0, 10):
#         obs_test, info_test = env_testing.reset(seed=2024)
        
#         # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))

#         total_reward = 0
#         done_test = False
#         env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(model_nbr)), :].index)
#         obs_training, info_training = env_train.reset(seed=2024)
#         model_ppo = PPO.load(f'models\model_{model_nbr}.pkl', env=env_train)

#         while not done_test:
#             action, _states = model_ppo.predict(obs_test)
#             obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#             done_test = terminated_test or truncated_test

#             total_reward += reward_test
#             if done_test:
#                 break
#         over_episodes_balance.append(info_test['balance'])
#         over_episodes_rewards.append(total_reward)
#         print(f'Episode: {episode}, Reward: {total_reward:.3f}, Balance: {info_test["balance"]:.3f}')
#     print_stats(over_episodes_rewards)
#     print_stats(over_episodes_balance)

In [27]:
# data = pd.DataFrame(plot_data)

# sns.set_style('whitegrid')
# plt.figure(figsize=(8, 6))

# for key in plot_data:
#     if key == 'x':
#         continue
#     label = plot_settings[key]['label']
#     line = plt.plot('x', key, data=data, linewidth=1, label=label)

# plt.xlabel('episode')
# plt.ylabel('reward')
# plt.title('Random vs. SB3 Agents')
# plt.legend()
# plt.show()